# Business Problem Statement: 
- Everday (EOD) one file we will receive 
- My business works monday to friday
- Saturday we need to load data into mongo database
- Once loading is completed take the backup
- Send mails to all stack hoders

## Step 1. Importing Required Modules

In [1]:
import os
import glob
import pandas as pd

# mail related modules
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# mongo module
from pymongo import MongoClient

path="C:\\Users\\ramreddymyla\\Google Drive\\01 DS ML DL NLP and AI With Python Lab Copy\\02 Lab Data\\Python\\files_automation"
os.chdir(path)
import shutil

# To work with dates
from datetime import datetime
from time import sleep

# it is useful to get version info...etc
import sys

# import logging module
import logging
logging.basicConfig(filename="logfile.log", level=logging.INFO)

## 2. Searching for Files

In [2]:
while True:    
    file_count=len(glob.glob("transaction*.csv"))
    logging.info(file_count)    
    logging.info(f'searching for files {file_count} started at {datetime.now().ctime()}')
    if file_count == 5:
        logging.info(f'Recived all {file_count} Files')
        files_cv  = glob.glob("transaction*.csv") 
        logging.info(f'File Names are {files_cv} ')        
        break        
    else:
        logging.info(f'Files not found waiting for files {datetime.now().ctime()}')
        sleep(20)

## 3. Connect to Mongo Database

In [3]:
try: 
    conn = MongoClient() 
    db = conn.rritec12oct2019
    # connecting to table
    collection = db.customers
    logging.info(f'Mongo Database Connected successfully and connection details are {collection}')
except:   
    #print("Could not connect to MongoDB")
    logging.info('Error while coonecting to Mongo Database')
    

## 4. Load All Files to Mongo Database

In [4]:
for csv in files_cv:
    
    # reading the csv file        
    df = pd.read_csv(csv) 
    
    # updating recent data
    collection.insert_many(df.to_dict('records'))        
logging.info(f'Inserted All {file_count} files')        

## 5. Backup All Files

In [6]:
#name="files_"+datetime.today().strftime('%Y%m%d')
#path = os.getcwd()
#shutil.make_archive(name,'zip',path)
#logging.info('creating backup for files')

## 6. Send Mail To Business/Customers/Down Stream App owners

In [5]:
mail_list = ["dwramreddy@gmail.com","rritec@gmail.com"]
msg=MIMEMultipart()
message_body=f"Mongo Data Refresh Completed {datetime.now().ctime()}"
msg.attach(MIMEText(message_body,'plain'))
#msg.attach(MIMEText(open("logfile.log").read()))


In [6]:
#Note: get app password by following and place in below txt file
    
#https://support.teamgate.com/hc/en-us/articles/115002064229-How-to-create-a-password-to-connect-email-while-using-2-step-verification-in-Gmail-

In [7]:
for i in range(len(mail_list)):
    s = smtplib.SMTP('smtp.gmail.com',587)
    s.starttls()
    s.login("dwramreddy@gmail.com", open("E:\password.txt").read())
    msg['Subject']= f"Files updated successfully {datetime.now().ctime()}"
    s.sendmail("dwramreddy@gmail.com", mail_list[i], msg.as_string())
    s.quit()

In [10]:
# try scheduling using windows machine 
# note that in reality we will use linux crontab or third party softwares like oozie to schedulr

# https://www.esri.com/arcgis-blog/products/product/analytics/scheduling-a-python-script-or-model-to-run-at-a-prescribed-time/
    